En este notebook se mostrará el código referente al entrenamiento del modelo, así como del análisis de resultados y ejecución del programa con el modelo generado.

En primer lugar, se muestra el código utilizado para el entrenamiento de nuestro modelo:

In [ ]:
# Importaciones
!pip3 install peft
!pip3 install scikit-learn
!pip3 install datasets

import torch
import os
import pandas as pd
import numpy as np
import peft
from transformers import AutoConfig, AutoModelForSequenceClassification, AutoTokenizer, DataCollatorWithPadding, Trainer, TrainingArguments, pipeline
from peft import LoraConfig, PeftModel
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.preprocessing import MultiLabelBinarizer
from datasets import load_dataset, Dataset, DatasetDict
from huggingface_hub import snapshot_download

Realizadas las importaciones, se procede a la descarga del modelo pre-entrenado especializado en clasificación de secuencias

In [2]:
nombre_modelo = 'PlanTL-GOB-ES/roberta-large-bne-massive'
snapshot_download(repo_id=nombre_modelo, cache_dir='./huggingface_mirror')

Se procede a la apertura del DataFrame con los datos de entrenamiento y se generan las etiquetas, que serán las clases del modelo supervisado, y sus números asociados, pues los Transformers entienden números, no textos:

In [21]:
def generar_lista_etiquetas(fila):
    if type(fila['acciones']) is float:
        return ''
        
    # Se añaden, en primer lugar, las acciones
    lista = fila['acciones'].split(',')

    # Se añade a continuación el YAML, que pueden ser una prueba o un conjunto de ellas
    if fila['YAML'].startswith('['):
        yamls = fila['YAML'][1:-1]
        yamls = yamls.split('URL:')
        yamls = ['URL:' + yaml for yaml in yamls]
        lista = lista + yamls
        
    else:
        lista.append(fila['YAML'])    

    while 'URL:\n' in lista:
        lista.remove('URL:\n')
        
    return lista

def cambiar_a_float(etiquetas):
    etiquetas = [float(etiqueta) for etiqueta in etiquetas]
    return etiquetas
    
nombre_fichero = os.getcwd() + '\\ficheros_excel\\MODELO_ENTRENAMIENTO.xlsx'
df_modelo_entrenamiento = pd.read_excel(io=nombre_fichero, sheet_name='MODELO_ENTRENAMIENTO', index_col=None)
df_modelo_entrenamiento = df_modelo_entrenamiento[['Test', 'YAML', 'acciones']]

# Se renombran las columnas, ya que de lo contrario puede dar errores durante el entrenamiento
df_modelo_entrenamiento.columns = ['text', 'YAML', 'acciones']

# Se genera una nueva columna, que contendrá una lista con el YAML y las acciones de manera conjunta para, posteriormente, crear etiquetas
df_modelo_entrenamiento['lista_etiquetas'] = df_modelo_entrenamiento.apply(generar_lista_etiquetas, axis=1)

df_modelo_entrenamiento = df_modelo_entrenamiento[['text', 'lista_etiquetas']]

# Se generan nuevas columnas, que serán las diversas etiquetas y que podrán tener un valor de 0 o 1, en función de si para el texto o tributo de entrada se corresponde o no dicha etiqueta
binarizador_multietiqueta = MultiLabelBinarizer()
one_hot_labels = binarizador_multietiqueta.fit_transform(df_modelo_entrenamiento['lista_etiquetas'])
df_etiquetas_codificadas = pd.DataFrame(one_hot_labels, columns=binarizador_multietiqueta.classes_)

df_modelo_entrenamiento = pd.concat([df_modelo_entrenamiento['text'], df_etiquetas_codificadas], axis=1)

# Dado que el trasnformer únicamente entiende valores numéricos, se proceden a crear diccionarios que contendrá el valor numérico correspondiente a cada etiqueta de texto
etiquetas = df_modelo_entrenamiento.columns.tolist()
etiquetas.remove('text')
numero_de_etiquetas = len(etiquetas)
ids_a_etiquetas = {idx:label for idx, label in enumerate(etiquetas)}
etiquetas_a_id ={label:idx for idx, label in enumerate(etiquetas)}

# Se introducen los valores en una lista, que posteriormente se convertirán a valores flotantes
df_modelo_entrenamiento['labels'] = df_modelo_entrenamiento[etiquetas].values.tolist()
df_modelo_entrenamiento['labels'] = df_modelo_entrenamiento['labels'].apply(cambiar_a_float)

# Se eliminan las columnas independientes referentes a las etiquetas
df_modelo_entrenamiento.drop(columns=etiquetas, inplace=True)

print(df_modelo_entrenamiento.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1286 entries, 0 to 1285
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    1286 non-null   object
 1   labels  1286 non-null   object
dtypes: object(2)
memory usage: 20.2+ KB
None


Para adaptar el modelo pre-entrenado al que se entrenará, resulta necesaria la modificación de la capa de clasificación, para lo cual se obtendrá la configuración del modelo preentrenado:

In [22]:
# Creamos la configuración para modificar la capa de clasificación de nuestro modelo preentrenado al de nuestro modelo
nueva_configuracion_modelo = AutoConfig.from_pretrained(nombre_modelo, num_labels=numero_de_etiquetas, id2label=ids_a_etiquetas, label2id=etiquetas_a_id, cache_dir='./huggingface_mirror', problem_type='multi_label_classification')

Se obtienen a continuación el tokenizador y modelo ya pre-entrenados:

In [34]:
# Obtenemos el modelo y tokenizador del modelo ya preentrenado
modelo_roberta = AutoModelForSequenceClassification.from_pretrained('PlanTL-GOB-ES/roberta-large-bne-massive', problem_type='multi_label_classification', cache_dir='./huggingface_mirror', local_files_only=True)
print(modelo_roberta)

# Dado que el número de clases del clasificador del modelo a entrenar son distintos del modelo pre-entrenado a utilizar, se ha de modificar la capa out_proj
modelo_roberta.classifier.out_proj = torch.nn.Linear(modelo_roberta.classifier.out_proj.in_features, numero_de_etiquetas, bias=True)
modelo_roberta.num_labels = numero_de_etiquetas
modelo_roberta.config = nueva_configuracion_modelo

print('--------------------------')
print(modelo_roberta)

tokenizador_roberta = AutoTokenizer.from_pretrained(nombre_modelo, cache_dir='./huggingface_mirror', local_files_only=True, from_pt=True)

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50262, 1024, padding_idx=1)
      (position_embeddings): Embedding(514, 1024, padding_idx=1)
      (token_type_embeddings): Embedding(1, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-23): 24 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSdpaSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=Tru

Se procede a continuación a la carga del dataset, separando los datos para entrenamiento, test y validación. Una vez hecho, se hará la tokenización de los atributos, es decir, se transformarán las palabras en números o, dicho de otro modo, datos entendibles por la arquitectura utilizada para el entrenamiento:

In [27]:
def tokenizador_texto(textos, tokenizador):
    texto = textos['text']
    textos_tokenizados = tokenizador(texto, return_tensors='pt', truncation=True, padding='max_length', max_length=512)
    
    return textos_tokenizados

dataset = Dataset.from_pandas(df_modelo_entrenamiento, split='train')
dataset = dataset.train_test_split(shuffle=True, seed=42, test_size=0.2)
print(dataset)

if tokenizador_roberta.pad_token is None:
    tokenizador_roberta.add_special_tokens({'pad_token': '[PAD]'})
    modelo_roberta.resize_token_embeddings(len(tokenizador_roberta))

dataset_tokenizado = dataset.map(tokenizador_texto, batched=True, fn_kwargs={'tokenizador': tokenizador_roberta})

DatasetDict({
    train: Dataset({
        features: ['text', 'labels'],
        num_rows: 1028
    })
    test: Dataset({
        features: ['text', 'labels'],
        num_rows: 258
    })
})


Map: 100%|██████████████████████████████████████████████████████████████████| 258/258 [00:00<00:00, 2956.81 examples/s]


Se genera a continuación un modelo PEFT mediante LoRA. Esto nos permite la congelación de los pesos del modelo pre-entrenado que, además de reducir los tiempos de entrenamiento, facilita unos mejores resultados cuando los datos de entrenamiento son escasos como aquí sucede.

In [28]:
# En la siguiente línea, se busca realizar padding en todas las secuencias, de manera que todas coincidan en longitud con la más larga. Esto es de utilizad para conseguir una mayor eficiencia
data_collator = DataCollatorWithPadding(tokenizer=tokenizador_roberta, padding=True)

configuracion_peft = LoraConfig(task_type='SEQ_CLS', r=4, lora_alpha=32, lora_dropout=0.01, target_modules=['query'])
modelo_peft = peft.get_peft_model(model=modelo_roberta, peft_config=configuracion_peft)

# Se muestran las cantidades de parámetros, siendo dichos parámetros el total, el número de parámetros entrenables y qué porcentaje de éstos se entrenará
modelo_peft.print_trainable_parameters()

trainable params: 1,631,608 || all params: 357,373,680 || trainable%: 0.4566


Se configuran en la siguiente celda los hiperparámetros y otras variables de entrenamiento:

In [32]:
def obtener_metricas(p):
    print(p)
    predicciones, etiquetas = p
    etiquetas = np.argmax(etiquetas, axis=1)
    predicciones = np.argmax(predicciones, axis=1)
    accuracy = accuracy_score(etiquetas, predicciones)
    precision = precision_score(etiquetas, predicciones, average='weighted')
    recall = recall_score(etiquetas, predicciones, average='weighted')
    f1 = f1_score(etiquetas, predicciones, average='weighted')
    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}

training_args = TrainingArguments(
    output_dir=nombre_modelo + '-custom-lora',
    learning_rate=3e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
    eval_strategy='epoch',
    save_strategy='epoch',
    load_best_model_at_end=True,
    optim='adamw_torch',
    push_to_hub=False
)

trainer = Trainer(
    model=modelo_peft,
    args=training_args,
    train_dataset=dataset_tokenizado['train'],
    eval_dataset=dataset_tokenizado['test'],
    tokenizer=tokenizador_roberta,
    data_collator=data_collator,
    compute_metrics=obtener_metricas
)

C:\Users\jmarrieta\AppData\Local\Temp\ipykernel_26292\2424049987.py:26: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Se procede ahora al entrenamiento, guardando el modelo tras finalización, y a la evaluación de resultados:

In [33]:
trainer.train()
modelo_peft.to('cpu')
modelo_peft.eval()
modelo_peft.save_pretrained(nombre_modelo + '_peft')
trainer.save_model()

C:\Users\jmarrieta\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\utils\data\dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,No log,0.033693,0.000000,0.000000,0.000000,0.000000
2,No log,0.032943,0.000000,0.000000,0.000000,0.000000
3,No log,0.032610,0.000000,0.000000,0.000000,0.000000


C:\Users\jmarrieta\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:98: UserWarning: The number of unique classes is greater than 50% of the number of samples. `y` could represent a regression problem, not a classification problem.
  type_true = type_of_target(y_true, input_name="y_true")
C:\Users\jmarrieta\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:98: UserWarning: The number of unique classes is greater than 50% of the number of samples. `y` could represent a regression problem, not a classification problem.
  type_true = type_of_target(y_true, input_name="y_true")
C:\Users\jmarrieta\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\multiclass.py:79: UserWarning: The number of unique classes is greater than 50% of the number of samples. `y` could represent a regression problem, not a classification problem.
  ys_types = set(type_of_target(x) for x in ys)
C:\Users\jmarri

C:\Users\jmarrieta\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:98: UserWarning: The number of unique classes is greater than 50% of the number of samples. `y` could represent a regression problem, not a classification problem.
  type_true = type_of_target(y_true, input_name="y_true")
C:\Users\jmarrieta\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:98: UserWarning: The number of unique classes is greater than 50% of the number of samples. `y` could represent a regression problem, not a classification problem.
  type_true = type_of_target(y_true, input_name="y_true")
C:\Users\jmarrieta\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\multiclass.py:79: UserWarning: The number of unique classes is greater than 50% of the number of samples. `y` could represent a regression problem, not a classification problem.
  ys_types = set(type_of_target(x) for x in ys)
C:\Users\jmarri

C:\Users\jmarrieta\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:98: UserWarning: The number of unique classes is greater than 50% of the number of samples. `y` could represent a regression problem, not a classification problem.
  type_true = type_of_target(y_true, input_name="y_true")
C:\Users\jmarrieta\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:98: UserWarning: The number of unique classes is greater than 50% of the number of samples. `y` could represent a regression problem, not a classification problem.
  type_true = type_of_target(y_true, input_name="y_true")
C:\Users\jmarrieta\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\multiclass.py:79: UserWarning: The number of unique classes is greater than 50% of the number of samples. `y` could represent a regression problem, not a classification problem.
  ys_types = set(type_of_target(x) for x in ys)
C:\Users\jmarri

KeyboardInterrupt: 

En el siguiente apartado se visualizará la matriz de confusión, de utilidad para conocer las clases cuya predicción coincide con la realidad y aquellas en las que no, dividiéndose en Verdaderos Positivos, Falsos Positivos, Verdaderos Negativos y Falsos Negativos

In [ ]:
def generar_matriz_de_confusion(y_verdadero, y_predicho):
    # Se crea la matriz de confusión
    matriz_de_confusion = confusion_matrix(y_verdadero, y_predicho)

    # Se crea la figura y se muestra la matriz de confusión mediante un mapa de calor
    plt.figure(fig_size=(20,10))
    sns.heatmap(matriz_de_confusion, annot=True, fmt='d', cmap='blues', xticklabels=etiquetas, yticklabels=etiquetas)
    plt.title('Matriz de confusión')
    plt.xlabel('Etiqueta predicha')
    plt.ylabel('Etiqueta real')
    plt.show()

generar_matriz_de_confusion(etiquetas, predicciones)

Tras finalización de entrenamiento del modelo y visualización de resultados, se procede a la escritura del código que permitirá cargar el modelo ya entrenado para su uso. En primer lugar, se hará una unión del modelo original y el modelo PEFT.

In [ ]:
# Se cargan ambos modelos
modelo_entrenado = AutoModelForSequenceClassification.from_pretrained(nombre_modelo + '-custom-lora', cache_dir='./huggingface_mirror', local_files_only=True)
modelo_peft = PeftModel.from_pretrained(nombre_modelo + '_peft')

# Se procede a la unión y guardado
modelo_peft = modelo_peft.merge_and_unload()
modelo_peft.save_pretrained(nombre_modelo + '_modelo_final')

 Se abre el modelo final para su uso

In [ ]:
#modelo = PeftModel.from_pretrained(model=nombre_modelo + '_peft', model_id=)

secuenciador = pipeline('text-classification', nombre_modelo + '_peft')

resultado = secuenciador('pruebas histograma auxiliar')